In [3]:
import time
import mnist
import numpy as np

from sklearn.tree import DecisionTreeClassifier

from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO
from IPython.display import Image
import pydotplus

from keras.utils import np_utils
from keras import Sequential
from keras import layers
    

In [4]:
# gets the data
def get_data():
    test = mnist.test_images()
    label_test = mnist.test_labels()
    train = mnist.train_images()
    label_train = mnist.train_labels()
    
    nsamples, nx, ny = train.shape
    train = train.reshape((nsamples,nx*ny))
    
    nsamples, nx, ny = test.shape
    test = test.reshape((nsamples,nx*ny))
    
    return train, label_train, test, label_test

In [5]:
from sklearn.tree._tree import TREE_LEAF

def prune_index(inner_tree, index, threshold):
    if inner_tree.value[index].min() < threshold:
        # turn node into a leaf by "unlinking" its children
        inner_tree.children_left[index] = TREE_LEAF
        inner_tree.children_right[index] = TREE_LEAF
    # if there are shildren, visit them as well
    if inner_tree.children_left[index] != TREE_LEAF:
        prune_index(inner_tree, inner_tree.children_left[index], threshold)
        prune_index(inner_tree, inner_tree.children_right[index], threshold)

In [6]:
# builds the decision tree of depth 10
def decision_tree(train, label):
    dt = DecisionTreeClassifier(max_depth = 10, random_state = 1)
    dt.fit(train, label)
    prune_index(dt.tree_, 0, 1)
    return dt

In [7]:
# building the neural network
def neural_network(class_data):
    num_train = []
    num_label = []
    for x in class_data:
        num_train.append(x[0])
        num_label.append(x[1])
    
    n_classes = 10
    num_train = np.matrix(num_train).astype('float32')/255
    num_label = np.array(num_label)
    num_label = np_utils.to_categorical(num_label, n_classes)
    
    model = Sequential()
    model.add(layers.Dense(512, activation = 'relu', input_shape=(784,)))
    model.add(layers.Dense(512, activation = 'relu'))
    model.add(layers.Dense(10, activation = 'softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], \
                  optimizer='adam')
    model.fit(num_train, num_label, batch_size=128, epochs=10)
    return model

In [8]:
# builds the neural shrub
def neural_shrubs(tree, train, label):
    train = np.array(train)
    label = np.array(label)
  
    # leave_id: index of the leaf that cantains the instance
    leave_id = tree.apply(train) 

    classes = dict()

    for x in range(len(train)):
        leaf = leave_id[x]
        
        # Gets the class for each leaf
        #.value: contains value of all the tree nodes
        #.value[leaf]: returns the value of the leaf
        #idx = tree.tree_.value[leaf][0][0]
        
        # insert the instance into the class
        if leaf in classes.keys():
            classes[leaf].append([train[x], label[x]])
        else:
            classes[leaf] = [[train[x], label[x]]]
    
    # stores the neural network for each class
    nn_models = dict()
    
    #stores the max time taken to build a neural network
    max_time = 0;

    for key in classes.keys():
        
        start = time.time()
        model = neural_network(classes[key])
        end = time.time()
        
        time_taken = end - start
        if max_time < time_taken:
            max_time = time_taken
        
        nn_models[key] = model

    # returns a neural network for each class and the max 
    # time taken to build the neural network
    return nn_models, max_time

In [9]:
# The algorithm to build the neural shrub
train, train_label, test, test_label = get_data()

dt_start = time.time()

tree = decision_tree(train, train_label)
dot_data = StringIO()
export_graphviz(tree, out_file=dot_data)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

dt_end = time.time()


In [10]:
shrubs, max_time = neural_shrubs(tree, train, train_label)

Epoch 1/10
368/368 [==============================] - 0s 1ms/step - loss: 1.4510 - acc: 0.5761
Epoch 2/10
368/368 [==============================] - 0s 153us/step - loss: 0.6790 - acc: 0.8641
Epoch 3/10
368/368 [==============================] - 0s 157us/step - loss: 0.4023 - acc: 0.8750
Epoch 4/10
368/368 [==============================] - 0s 142us/step - loss: 0.2787 - acc: 0.9293
Epoch 5/10
368/368 [==============================] - 0s 138us/step - loss: 0.1959 - acc: 0.9538
Epoch 6/10
368/368 [==============================] - 0s 137us/step - loss: 0.1413 - acc: 0.9674
Epoch 7/10
368/368 [==============================] - 0s 145us/step - loss: 0.1117 - acc: 0.9728
Epoch 8/10
368/368 [==============================] - 0s 141us/step - loss: 0.0747 - acc: 0.9837
Epoch 9/10
368/368 [==============================] - 0s 158us/step - loss: 0.0548 - acc: 0.9891
Epoch 10/10
368/368 [==============================] - 0s 153us/step - loss: 0.0411 - acc: 0.9918
Epoch 1/10
476/476 [===========

662/662 [==============================] - 0s 158us/step - loss: 0.2249 - acc: 0.9290
Epoch 6/10
662/662 [==============================] - 0s 162us/step - loss: 0.1706 - acc: 0.9381
Epoch 7/10
662/662 [==============================] - 0s 153us/step - loss: 0.1077 - acc: 0.9728
Epoch 8/10
662/662 [==============================] - 0s 161us/step - loss: 0.0838 - acc: 0.9804
Epoch 9/10
662/662 [==============================] - 0s 174us/step - loss: 0.0651 - acc: 0.9819
Epoch 10/10
662/662 [==============================] - 0s 148us/step - loss: 0.0468 - acc: 0.9909
Epoch 1/10
148/148 [==============================] - 1s 4ms/step - loss: 2.1050 - acc: 0.2635
Epoch 2/10
148/148 [==============================] - 0s 236us/step - loss: 1.1436 - acc: 0.5811
Epoch 3/10
148/148 [==============================] - 0s 228us/step - loss: 0.8333 - acc: 0.7703
Epoch 4/10
148/148 [==============================] - 0s 254us/step - loss: 0.6596 - acc: 0.7770
Epoch 5/10
148/148 [======================

Epoch 1/10
197/197 [==============================] - 1s 4ms/step - loss: 2.0770 - acc: 0.2538
Epoch 2/10
197/197 [==============================] - 0s 198us/step - loss: 0.9584 - acc: 0.7411
Epoch 3/10
197/197 [==============================] - 0s 170us/step - loss: 0.8293 - acc: 0.7462
Epoch 4/10
197/197 [==============================] - 0s 224us/step - loss: 0.5976 - acc: 0.8274
Epoch 5/10
197/197 [==============================] - 0s 223us/step - loss: 0.5244 - acc: 0.8832
Epoch 6/10
197/197 [==============================] - 0s 215us/step - loss: 0.3846 - acc: 0.8832
Epoch 7/10
197/197 [==============================] - 0s 179us/step - loss: 0.2999 - acc: 0.8934
Epoch 8/10
197/197 [==============================] - 0s 190us/step - loss: 0.2479 - acc: 0.9340
Epoch 9/10
197/197 [==============================] - 0s 212us/step - loss: 0.1904 - acc: 0.9543
Epoch 10/10
197/197 [==============================] - 0s 209us/step - loss: 0.1467 - acc: 0.9746
Epoch 1/10
81/81 [=============

975/975 [==============================] - 0s 129us/step - loss: 0.0499 - acc: 0.9856
Epoch 6/10
975/975 [==============================] - 0s 151us/step - loss: 0.0354 - acc: 0.9867
Epoch 7/10
975/975 [==============================] - 0s 125us/step - loss: 0.0232 - acc: 0.9949
Epoch 8/10
975/975 [==============================] - 0s 137us/step - loss: 0.0164 - acc: 0.9959
Epoch 9/10
975/975 [==============================] - 0s 125us/step - loss: 0.0106 - acc: 0.9969
Epoch 10/10
975/975 [==============================] - 0s 150us/step - loss: 0.0061 - acc: 0.9990
Epoch 1/10
72/72 [==============================] - 1s 13ms/step - loss: 2.3173 - acc: 0.1111
Epoch 2/10
72/72 [==============================] - 0s 255us/step - loss: 1.7062 - acc: 0.4861
Epoch 3/10
72/72 [==============================] - 0s 246us/step - loss: 1.2921 - acc: 0.6944
Epoch 4/10
72/72 [==============================] - 0s 257us/step - loss: 0.9498 - acc: 0.9167
Epoch 5/10
72/72 [==============================]

1282/1282 [==============================] - 0s 150us/step - loss: 0.0170 - acc: 0.9984
Epoch 1/10
64/64 [==============================] - 1s 17ms/step - loss: 2.2725 - acc: 0.1094
Epoch 2/10
64/64 [==============================] - 0s 251us/step - loss: 1.7075 - acc: 0.4375
Epoch 3/10
64/64 [==============================] - 0s 233us/step - loss: 1.4071 - acc: 0.5156
Epoch 4/10
64/64 [==============================] - 0s 277us/step - loss: 1.1934 - acc: 0.5469
Epoch 5/10
64/64 [==============================] - 0s 237us/step - loss: 0.9757 - acc: 0.6562
Epoch 6/10
64/64 [==============================] - 0s 279us/step - loss: 0.7787 - acc: 0.8125
Epoch 7/10
64/64 [==============================] - 0s 257us/step - loss: 0.6111 - acc: 0.8281
Epoch 8/10
64/64 [==============================] - 0s 265us/step - loss: 0.4719 - acc: 0.8906
Epoch 9/10
64/64 [==============================] - 0s 429us/step - loss: 0.3594 - acc: 0.9375
Epoch 10/10
64/64 [==============================] - 0s 29

...

164/164 [==============================] - 0s 232us/step - loss: 0.2372 - acc: 0.9573
Epoch 8/10
164/164 [==============================] - 0s 252us/step - loss: 0.1860 - acc: 0.9695
Epoch 9/10
164/164 [==============================] - 0s 203us/step - loss: 0.1433 - acc: 0.9756
Epoch 10/10
164/164 [==============================] - 0s 239us/step - loss: 0.1131 - acc: 0.9756
Epoch 1/10
66/66 [==============================] - 2s 36ms/step - loss: 2.3501 - acc: 0.0606
Epoch 2/10
66/66 [==============================] - 0s 290us/step - loss: 1.4976 - acc: 0.5303
Epoch 3/10
66/66 [==============================] - 0s 209us/step - loss: 1.2163 - acc: 0.5303
Epoch 4/10
66/66 [==============================] - 0s 262us/step - loss: 1.0607 - acc: 0.6818
Epoch 5/10
66/66 [==============================] - 0s 341us/step - loss: 0.9282 - acc: 0.7424
Epoch 6/10
66/66 [==============================] - 0s 251us/step - loss: 0.8112 - acc: 0.7424
Epoch 7/10
66/66 [==============================] - 0

133/133 [==============================] - 3s 20ms/step - loss: 2.3672 - acc: 0.0526
Epoch 2/10
133/133 [==============================] - 0s 272us/step - loss: 1.2906 - acc: 0.5113
Epoch 3/10
133/133 [==============================] - 0s 274us/step - loss: 1.1078 - acc: 0.6466
Epoch 4/10
133/133 [==============================] - 0s 283us/step - loss: 0.9558 - acc: 0.7744
Epoch 5/10
133/133 [==============================] - 0s 349us/step - loss: 0.7774 - acc: 0.7895
Epoch 6/10
133/133 [==============================] - 0s 292us/step - loss: 0.5920 - acc: 0.8496
Epoch 7/10
133/133 [==============================] - 0s 293us/step - loss: 0.5135 - acc: 0.8872
Epoch 8/10
133/133 [==============================] - 0s 331us/step - loss: 0.4644 - acc: 0.8797
Epoch 9/10
133/133 [==============================] - 0s 283us/step - loss: 0.3709 - acc: 0.8947
Epoch 10/10
133/133 [==============================] - 0s 277us/step - loss: 0.2689 - acc: 0.9248
Epoch 1/10
3833/3833 [===================

148/148 [==============================] - 0s 244us/step - loss: 0.6409 - acc: 0.8581
Epoch 6/10
148/148 [==============================] - 0s 259us/step - loss: 0.5097 - acc: 0.8649
Epoch 7/10
148/148 [==============================] - 0s 253us/step - loss: 0.3850 - acc: 0.8784
Epoch 8/10
148/148 [==============================] - 0s 265us/step - loss: 0.2852 - acc: 0.9257
Epoch 9/10
148/148 [==============================] - 0s 342us/step - loss: 0.2508 - acc: 0.9324
Epoch 10/10
148/148 [==============================] - 0s 270us/step - loss: 0.1909 - acc: 0.9527
Epoch 1/10
31/31 [==============================] - 3s 91ms/step - loss: 2.3930 - acc: 0.0000e+00
Epoch 2/10
31/31 [==============================] - 0s 729us/step - loss: 1.5037 - acc: 0.7097
Epoch 3/10
31/31 [==============================] - 0s 535us/step - loss: 0.9873 - acc: 0.7097
Epoch 4/10
31/31 [==============================] - 0s 528us/step - loss: 0.7906 - acc: 0.7097
Epoch 5/10
31/31 [===========================

1707/1707 [==============================] - 0s 158us/step - loss: 0.0029 - acc: 1.0000
Epoch 1/10
84/84 [==============================] - 3s 37ms/step - loss: 2.1085 - acc: 0.5119
Epoch 2/10
84/84 [==============================] - 0s 315us/step - loss: 1.3004 - acc: 0.6905
Epoch 3/10
84/84 [==============================] - 0s 258us/step - loss: 1.1051 - acc: 0.6905
Epoch 4/10
84/84 [==============================] - 0s 276us/step - loss: 1.0260 - acc: 0.6905
Epoch 5/10
84/84 [==============================] - 0s 209us/step - loss: 0.8483 - acc: 0.6905
Epoch 6/10
84/84 [==============================] - 0s 276us/step - loss: 0.6750 - acc: 0.7738
Epoch 7/10
84/84 [==============================] - 0s 174us/step - loss: 0.5855 - acc: 0.8929
Epoch 8/10
84/84 [==============================] - 0s 211us/step - loss: 0.5312 - acc: 0.9167
Epoch 9/10
84/84 [==============================] - 0s 193us/step - loss: 0.4336 - acc: 0.9167
Epoch 10/10
84/84 [==============================] - 0s 22

Epoch 6/10
62/62 [==============================] - 0s 412us/step - loss: 0.4309 - acc: 0.8871
Epoch 7/10
62/62 [==============================] - 0s 262us/step - loss: 0.3215 - acc: 0.9032
Epoch 8/10
62/62 [==============================] - 0s 281us/step - loss: 0.2243 - acc: 0.9194
Epoch 9/10
62/62 [==============================] - 0s 241us/step - loss: 0.1555 - acc: 0.9677
Epoch 10/10
62/62 [==============================] - 0s 344us/step - loss: 0.1202 - acc: 0.9839
Epoch 1/10
3818/3818 [==============================] - 4s 1ms/step - loss: 0.3479 - acc: 0.9442
Epoch 2/10
3818/3818 [==============================] - 1s 144us/step - loss: 0.1418 - acc: 0.9657
Epoch 3/10
3818/3818 [==============================] - 1s 145us/step - loss: 0.0872 - acc: 0.9733
Epoch 4/10
3818/3818 [==============================] - 1s 151us/step - loss: 0.0487 - acc: 0.9874
Epoch 5/10
3818/3818 [==============================] - 1s 149us/step - loss: 0.0240 - acc: 0.9937
Epoch 6/10
3818/3818 [=========

In [11]:
# predicts using the neural shrub
def neural_shrub_predict(tree, nn_model, test, label):
    label_test = np.array(label)
    test = np.array(test)
    
    #row - actual; col - pred
    confusion_matrix = np.zeros((10,10), dtype=np.int)
    correct = 0
    
    for i in range(len(test)):
        x = test[i]
        pred_class = tree.apply([x])
        x = np.array([x])
        nn_model_class = nn_model[pred_class[0]]
        pred = np.argmax(nn_model_class.predict(x))

        confusion_matrix[label[i]][pred] = \
            confusion_matrix[label[i]][pred] + 1
        if pred == label[i]: correct = correct + 1

    acc_score = correct/len(test)
    
    return confusion_matrix, acc_score

In [12]:
# function to calculate the metrics
def metrics(cm, cls, size):
    cm = np.array(cm)
    tp = cm[cls][cls]
    fp = sum(cm[x, cls] for x in range(10))-cm[cls][cls]
    fn = sum(cm[cls, x] for x in range(10))-cm[cls][cls]
    tn = size - tp - fp - fn
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    fmeasure = 2*(precision*recall)/(precision + recall)
    accuracy = (tp + tn)/size
    
    return precision, recall, fmeasure, accuracy

In [13]:
# Predicting
cm, acc_score = neural_shrub_predict(tree, shrubs, test, test_label)
print("Confusion Matrix:\n\n", cm)

Confusion Matrix:

 [[ 960    1    4    0    1    5    4    1    3    1]
 [   0 1117    4    4    0    1    4    0    5    0]
 [   6    0  970   13    5    3    8    8   18    1]
 [   4    0   14  941    1   26    1    5   13    5]
 [   1    2    1    1  920    2   12    5   12   26]
 [  10    1    2   22    2  814   14    2   20    5]
 [  12    3    3    2    6   10  915    1    6    0]
 [   1    8   15    1    4    2    0  980    5   12]
 [   1    0   16   16    5   16    3    3  904   10]
 [   4    2    3   10   19   11    0   13    7  940]]


In [14]:
# Class 0
precision0, recall0, f0, acc0 = metrics(cm, 0, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 0: ", round(precision0, 3), "  ", round(recall0, 3), \
      " ", round(f0, 3), "   ", round(acc0,3))

        Precision Recall F-measure Accuracy
Class 0:  0.961    0.98   0.97     0.994


In [15]:
# Class 1
precision1, recall1, f1, acc1 = metrics(cm, 1, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 1: ", round(precision1, 3), "  ", round(recall1, 3), \
      " ", round(f1, 3), "   ", round(acc1,3))

        Precision Recall F-measure Accuracy
Class 1:  0.985    0.984   0.985     0.996


In [16]:
# Class 2
precision2, recall2, f2, acc2 = metrics(cm, 2, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 2: ", round(precision2, 3), "  ", round(recall2, 3), \
      " ", round(f2, 3), "   ", round(acc2,3))

        Precision Recall F-measure Accuracy
Class 2:  0.94    0.94   0.94     0.988


In [17]:
# Class 3
precision3, recall3, f3, acc3 = metrics(cm, 3, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 3: ", round(precision3, 3), "  ", round(recall3, 3), \
      " ", round(f3, 3), "   ", round(acc3,3))

        Precision Recall F-measure Accuracy
Class 3:  0.932    0.932   0.932     0.986


In [18]:
# Class 4
precision4, recall4, f4, acc4 = metrics(cm, 4, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 4: ", round(precision4, 3), "  ", round(recall4, 3), \
      " ", round(f4, 3), "   ", round(acc4,3))

        Precision Recall F-measure Accuracy
Class 4:  0.955    0.937   0.946     0.99


In [19]:
# Class 5
precision5, recall5, f5, acc5 = metrics(cm, 5, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 5: ", round(precision5, 3), "  ", round(recall5, 3), \
      " ", round(f5, 3), "   ", round(acc5,3))

        Precision Recall F-measure Accuracy
Class 5:  0.915    0.913   0.914     0.985


In [20]:
# Class 5
precision5, recall5, f5, acc5 = metrics(cm, 5, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 5: ", round(precision5, 3), "  ", round(recall5, 3), \
      " ", round(f5, 3), "   ", round(acc5,3))

        Precision Recall F-measure Accuracy
Class 5:  0.915    0.913   0.914     0.985


In [21]:
# Class 6
precision6, recall6, f6, acc6 = metrics(cm, 6, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 6: ", round(precision6, 3), "  ", round(recall6, 3), \
      " ", round(f6, 3), "   ", round(acc6,3))

        Precision Recall F-measure Accuracy
Class 6:  0.952    0.955   0.954     0.991


In [22]:
# Class 7
precision7, recall7, f7, acc7 = metrics(cm, 7, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 7: ", round(precision0, 3), "  ", round(recall7, 3), \
      " ", round(f7, 3), "   ", round(acc7,3))

        Precision Recall F-measure Accuracy
Class 7:  0.961    0.953   0.958     0.991


In [23]:
# Class 8
precision8, recall8, f8, acc8 = metrics(cm, 8, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 8: ", round(precision8, 3), "  ", round(recall8, 3), \
      " ", round(f8, 3), "   ", round(acc8,3))

        Precision Recall F-measure Accuracy
Class 8:  0.91    0.928   0.919     0.984


In [24]:
# Class 9
precision9, recall9, f9, acc9 = metrics(cm, 9, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 9: ", round(precision9, 3), "  ", round(recall9, 3), \
      " ", round(f9, 3), "   ", round(acc9,3))

        Precision Recall F-measure Accuracy
Class 9:  0.94    0.932   0.936     0.987


In [25]:
# number of instances classified correctly
print("Accuracy_score: ", round(acc_score, 5))

Accuracy_score:  0.9461


In [26]:
# training time
total_time_taken = dt_end - dt_start + max_time
print("Training Time: %s sec" % round(total_time_taken, 5))

Training Time: 23.32264 sec
